In [1]:
import torch
import torchvision

from datasets.esri_datascience_challenge_2019 import ESRI_challenge_2019

from models.fastrcnn import FastRCNN
from models.retinanet import RetinaNet

In [2]:
%load_ext autoreload
%autoreload 2

## Define the dataset:

In [3]:
dataset = ESRI_challenge_2019 (root = "E:\\AiTLAS\\aitlas\\ESRI-challenge\\data\\ESRI",
                               subset = "train", 
                               subsample_percentage = 0.125, 
                               batch_size = 4, 
                               shuffle = True, 
                               num_workers = 0)

dataset_test = ESRI_challenge_2019 (root = "E:\\AiTLAS\\aitlas\\ESRI-challenge\\data\\ESRI",
                               subset = "test", 
                               subsample_percentage = 1, 
                               batch_size = 4, 
                               shuffle = True, 
                               num_workers = 0) 

The original number of images was: 3228
The subsampled number of images is: 378
The original number of images was: 520
The subsampled number of images is: 335


## Fine tuning on ESRI

In [4]:
def get_fastrcnn(num_classes = 3):
    model_cfg = {"num_classes": num_classes, "learning_rate": 0.001, "pretrained": True}

    # load a model pre-trained pre-trained on COCO
    model = FastRCNN(model_cfg)

    return model

def get_retinanet(num_classes = 3):
    model_cfg = {"num_classes": num_classes, "learning_rate": 0.001, "pretrained": True}
    model = RetinaNet(model_cfg)
    
    return model

### Move the model to the GPU and attach an optimizer as well as a learning rate scheduler:

In [5]:
# to train on gpu if selected.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 3

# get the model using our helper function
model = get_retinanet()

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [6]:
model.prepare()

In [7]:
model.train_and_evaluate_model(
    train_dataset=dataset,
    val_dataset=dataset_test,
    epochs=10,
    model_directory = "./experiment/",
    run_id = "1",
    iterations_log = 20
)

2021-06-23 18:30:27,959 INFO Starting training.
training: 100%|██████████| 95/95 [00:57<00:00,  1.66it/s]
2021-06-23 18:31:25,461 INFO epoch: 1, time: 58, loss:  1.19070
testing on train set: 100%|██████████| 95/95 [00:26<00:00,  3.61it/s]
2021-06-23 18:32:14,193 INFO Mean Average Precision:0.34957203
testing on validation set: 100%|██████████| 84/84 [00:22<00:00,  3.72it/s]
2021-06-23 18:34:03,173 INFO Mean Average Precision:0.23827769
training: 100%|██████████| 95/95 [00:52<00:00,  1.80it/s]
2021-06-23 18:34:55,964 INFO epoch: 2, time: 52, loss:  0.72559
testing on train set: 100%|██████████| 95/95 [00:26<00:00,  3.63it/s]
2021-06-23 18:35:59,025 INFO Mean Average Precision:0.48342994
testing on validation set: 100%|██████████| 84/84 [00:22<00:00,  3.66it/s]
2021-06-23 18:37:33,085 INFO Mean Average Precision:0.58526516
training: 100%|██████████| 95/95 [00:51<00:00,  1.83it/s]
2021-06-23 18:38:25,182 INFO epoch: 3, time: 52, loss:  0.51801
testing on train set: 100%|██████████| 95/95